# Converting from pixel coordinates in CZI files to stage coordinates

This notebook demonstrates how to extract pixel scale and stage position from Zeiss `.czi` files.
Using the pixel scale and stage position we build an affine transformation matrix that lets us convert
from pixel coordinates in a `.czi` image to absolute stage coordinates using simple matrix multiplication.

Volker.Hilsenstein@embl.de Oct 2017

In [1]:
from czitools import *

/Users/volkerhilsenstein/Dropbox/Volker/Work/GitHub/BioFormatsRead/czifile.py:130: UserWarning: failed to import the optional _czifile C extension module.
Decoding of JXR and JPEG encoded images will not be available.
Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _czifile C extension module.\n"


In [2]:
get_shapeinfo_cziread("/Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/single_capture.czi")

((1, 1, 2208, 2752, 1), b'BCYX0')

In [3]:
get_shapeinfo_cziread("/Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/5x5Acqu.czi")

((1, 1, 1, 10160, 12661, 1), b'BSCYX0')

In [4]:
pwd

'/Users/volkerhilsenstein/Dropbox/Volker/Work/GitHub/BioFormatsRead'

In [6]:
writexml_czi("/Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/single_capture.czi")

Write special CZI XML metainformation for:  /Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/single_capture_CZI_MetaData.xml


To find the stage Position we need to look for this stuff inside the XML metadata

```
     <ParameterCollection Id="MTBStageAxisX">
        <Position Status="Valid" Unit="µm">-102114.5</Position>
        <MeasurementPosition Status="Valid">-102114.5</MeasurementPosition>
        <IsPrecise Status="Valid">false</IsPrecise>
        <AxisSetPositionMode Status="Valid">Default</AxisSetPositionMode>
        <Speed Status="Valid">0</Speed>
        <ContinualSpeed Status="Valid">37500</ContinualSpeed>
        <ContinualAcceleration Status="Valid">0</ContinualAcceleration>
        <IsHandwheelDeactivated Status="Valid">false</IsHandwheelDeactivated>
        <IsLowerSWLimitEnabled Status="Valid">false</IsLowerSWLimitEnabled>
        <LowerSWLimit Status="Valid">0</LowerSWLimit>
        <IsUpperSWLimitEnabled Status="Valid">false</IsUpperSWLimitEnabled>
        <UpperSWLimit Status="Valid">0</UpperSWLimit>
        <AxisCalibrationMode Status="Valid">AutomaticallyCalibrated</AxisCalibrationMode>
      </ParameterCollection>
      <ParameterCollection Id="MTBStageAxisY">
        <Position Status="Valid" Unit="µm">-37692</Position>
        <MeasurementPosition Status="Valid">-37692</MeasurementPosition>
        <IsPrecise Status="Valid">false</IsPrecise>
        <AxisSetPositionMode Status="Valid">Default</AxisSetPositionMode>
        <Speed Status="Valid">0</Speed>

```


In [123]:
def get_metainfo_cziread_scaling(filename):
    # define default values in case something is missing inside the metadata
    xscale = np.NaN
    yscale = np.NaN
    xunit = "n.a."
    yunit = "n.a."
    xpos = (np.NaN, "n.a.")
    ypos = (np.NaN, "n.a.")

    try:
        czi = CziFile(filename)

        # Iterate over the metadata
        for elem in czi.metadata.getiterator():
            if 'Distance' in elem.tag or 'Scaling' in elem.tag:
                if repr(elem.attrib) == "{'Id': 'X'}":
                    xscale = float(elem.getchildren()[0].text)
                    xunit = elem.getchildren()[1].text
                if repr(elem.attrib) == "{'Id': 'Y'}":
                    yscale = float(elem.getchildren()[0].text)
                    yunit = elem.getchildren()[1].text
            
            if 'ParameterCollection' in elem.tag:
                if "MTBStageAxis" in elem.attrib['Id']:
                    children = elem.getchildren()
                    for c in children:
                        if c.tag == 'Position':
                            axispos = c.text
                            unit = c.attrib['Unit']
                        if 'X' in elem.attrib['Id']:
                            xpos = (float(axispos), unit)
                        elif 'Y'in elem.attrib['Id']:
                            ypos = (float(axispos), unit)
        czi.close()
    except:
        print('czifile.py did not detect an CZI file.')

    return (xscale, xunit), (yscale, yunit), xpos, ypos
    


In [124]:
elem = get_metainfo_cziread_scaling("/Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/5x5Acqu.czi")

# Bug in CZI Metadata: Note that unit reported for scale is _µm_ but actual value  of order e-07 indicated it is in fact _m_

In [125]:
elem

((1.135e-07, 'µm'), (1.135e-07, 'µm'), (-102114.5, 'µm'), (-37691.8, 'µm'))

# simple matrix multiplication in numpy

In [93]:
# matrix times vector
import numpy as np
a = np.array([[ 5, 1 ,3], 
                  [ 1, 1 ,1], 
                  [ 1, 2 ,1]])
b = np.array([1, 2, 3])
print (a.dot(b.transpose()))

[16  6  8]


In [95]:
# matrix times matrix (multiple vectors stacked)
import numpy as np
a = np.array([[ 5, 1 ,3], 
                  [ 1, 1 ,1], 
                  [ 1, 2 ,1]])
b = np.array([[1, 2, 3], [1,2,3]])
print (a.dot(b.transpose()).transpose())

[[16  6  8]
 [16  6  8]]


In [97]:
# Alternative syntax, but only works for Python >= 3.5
a @ b.transpose()


array([[16, 16],
       [ 6,  6],
       [ 8,  8]])

# Build affine transformation matrix to translate  from czi file

In [126]:
imagefileczi = "/Users/volkerhilsenstein/GitLab/zeiss-microbeam-feedback-microscopy/SampleImages/5x5Acqu.czi"
# get nx, ny
n_ax, axs = get_shapeinfo_cziread(imagefileczi)
nx = n_ax[axs.index(b'X')]
ny = n_ax[axs.index(b'Y')]
# get scale and stage position
xscale, yscale, xpos, ypos = get_metainfo_cziread_scaling(imagefileczi)



In [127]:
xscale, yscale

((1.135e-07, 'µm'), (1.135e-07, 'µm'))

In [149]:
mapping = np.array([[ xscale[0],    0 ,xpos[0] * 10**-6 -xscale[0]*nx/2.0], 
                    [ 0, yscale[0],ypos[0] * 10**-6 -yscale[0]*ny/2.0]])

In [150]:
mapping


array([[  1.13500000e-07,   0.00000000e+00,  -1.02833012e-01],
       [  0.00000000e+00,   1.13500000e-07,  -3.82683800e-02]])

In [152]:
mapping @np.array((nx/2, ny/2, 1.0))


array([-0.1021145, -0.0376918])

In [155]:
mapping @np.array([[100, 100, 1.0], [200,200, 1]]).transpose()

array([[-0.10282166, -0.10281031],
       [-0.03825703, -0.03824568]])